In [ ]:
# ! pip install 'crewai-tools[mcp]'

In [ ]:
import os
import sys
try:
    get_ipython
    current_dir = os.getcwd()
except NameError:
    current_dir = os.path.dirname(os.path.abspath(__file__))

# Set path，temporary path expansion
project_dir = os.path.abspath(os.path.join(current_dir, ''))
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [ ]:
import os
from crewai import Agent, Task, Crew,LLM
from crewai_tools import MCPServerAdapter
from mcp import StdioServerParameters  # 用于本地进程通信的 MCP 服务器参数
from dotenv import load_dotenv 

In [ ]:


load_dotenv()
BASE_URL= os.getenv("BASE_URL")
ONLINE_MODEL_NAME = os.getenv("ONLINE_MODEL_NAME")
api_key = ( os.getenv("QWEN_API_KEY") or os.getenv("DASHSCOPE_API_KEY" ) ) 

llm = LLM(
            base_url=BASE_URL,
            api_key=api_key,
            model="openai/"+ONLINE_MODEL_NAME,  # 本次使用的模型
            # temperature=0.7,  
            # timeout=None,
            # max_retries=2,
        )


In [ ]:


# 1. 配置 MCP 服务器参数（以本地 Stdio 服务器为例）
# 假设我们有一个本地 MCP 服务器脚本：calculator_server.py
server_path =os.path.join(project_dir,"calculator_server.py")
print(server_path)
server_params = StdioServerParameters(
    command="python3",
    args=[server_path],  # MCP 服务器脚本路径
    env={ **os.environ},  # 传递环境变量
)

# 2. 连接 MCP 服务器并获取工具
with MCPServerAdapter(server_params, connect_timeout=60) as mcp_tools:
    print(f"已加载 MCP 服务器提供的工具: {[tool.name for tool in mcp_tools]}")

    # 3. 创建使用 MCP 工具的 Agent
    calculator_agent = Agent(
        role="计算器专家",
        goal="使用 MCP 服务器提供的计算器工具解决数学问题",
        backstory="擅长调用外部计算工具解决复杂数学问题，依赖 MCP 服务器提供的计算能力",
        llm=llm,
        tools=mcp_tools,  # 注入 MCP 服务器的工具
        reasoning=True,
        verbose=True,
        max_iter=3
    )

    # 4. 创建任务
    calculation_task = Task(
        description="计算30的立方的结果",
        expected_output="详细计算步骤和最终结果，保留两位小数",
        agent=calculator_agent
    )

    # 5. 运行 Crew
    crew = Crew(agents=[calculator_agent], tasks=[calculation_task], verbose=True)
    result = crew.kickoff()

    print("\n任务结果:", result)
    